In [1]:
import pandas as pd
from collections import Counter, defaultdict
from pathlib import Path
import numpy as np

In [2]:
FILE_PATH = "../ml/moretoo/Ecommerce/train/df_Customers.csv"
OUTPUT_PATH = "../ml/moretoo/Ecommerce/train/cleaned_Customers.csv"
CHUNK_SIZE = 10000
MISSING_THRESHOLD = 0.95

In [3]:
col_missing_counts = Counter()
col_unique_ratios = {}
row_count = 0
total_chunks = 0

global_numeric_stats = defaultdict(list)
global_categorical_stats = defaultdict(Counter)

for chunk in pd.read_csv(FILE_PATH, chunksize=CHUNK_SIZE):
    row_count += len(chunk)
    total_chunks += 1
    
    col_missing_counts.update(chunk.isnull().sum().to_dict())
    
    for col in chunk.columns:
        unique_count = chunk[col].nunique()
        col_unique_ratios[col] = unique_count / len(chunk)
    
    if total_chunks <= 3:  
        numeric_cols = chunk.select_dtypes(include="number").columns
        categorical_cols = chunk.select_dtypes(include="object").columns
        
        for col in numeric_cols:
            non_null = chunk[col].dropna()
            if len(non_null) > 0:
                sample_size = min(100, len(non_null))
                global_numeric_stats[col].extend(non_null.sample(sample_size).tolist())
        
        for col in categorical_cols:
            non_null = chunk[col].dropna()
            if len(non_null) > 0:
                sample_size = min(50, len(non_null))
                global_categorical_stats[col].update(non_null.sample(sample_size))

print(f"Dataset analyzed: {row_count:,} rows, {len(col_missing_counts)} columns")
print(f" Total chunks: {total_chunks}")


Dataset analyzed: 89,316 rows, 4 columns
 Total chunks: 9


In [4]:
print("🧠 Smart column analysis...")

cols_to_drop = []
keep_reasons = {}

for col, missing_count in col_missing_counts.items():
    missing_ratio = missing_count / row_count
    unique_ratio = col_unique_ratios.get(col, 0)
    col_lower = col.lower()
    
    if missing_ratio >= MISSING_THRESHOLD:
        if any(keyword in col_lower for keyword in BUSINESS_KEYWORDS):
            keep_reasons[col] = f"Business metric (despite {missing_ratio:.1%} missing)"
            continue
        
        if unique_ratio > 0.8 and any(id_word in col_lower for id_word in ['id', 'key', 'code', 'ref']):
            keep_reasons[col] = f"Likely identifier (despite {missing_ratio:.1%} missing)"
            continue
        

        cols_to_drop.append(col)
    else:
        keep_reasons[col] = f"Good quality ({missing_ratio:.1%} missing)"

print(f" Column decisions:")
print(f" Dropping: {len(cols_to_drop)} columns")
print(f" Keeping: {len(keep_reasons)} columns")

🧠 Smart column analysis...
 Column decisions:
 Dropping: 0 columns
 Keeping: 4 columns


In [5]:
print("Examples of column decisions:")
print(" DROPPING:")
for i, col in enumerate(cols_to_drop[:5]):
    missing_ratio = col_missing_counts[col] / row_count
    print(f"   {i+1}. {col:<30} ({missing_ratio:.1%} missing)")

print("KEEPING:")
for i, (col, reason) in enumerate(list(keep_reasons.items())[:5]):
    print(f"   {i+1}. {col:<30} ({reason})")

if len(cols_to_drop) > 5:
    print(f"\n... and {len(cols_to_drop) - 5} more columns to drop")
if len(keep_reasons) > 5:
    print(f"... and {len(keep_reasons) - 5} more columns to keep")


Examples of column decisions:
 DROPPING:
KEEPING:
   1. customer_id                    (Good quality (0.0% missing))
   2. customer_zip_code_prefix       (Good quality (0.0% missing))
   3. customer_city                  (Good quality (0.0% missing))
   4. customer_state                 (Good quality (0.0% missing))


In [6]:
global_fallbacks = {}

for col, values in global_numeric_stats.items():
    if len(values) > 0:
        global_fallbacks[col] = pd.Series(values).median()

for col, counter in global_categorical_stats.items():
    if len(counter) > 0:
        global_fallbacks[col] = counter.most_common(1)[0][0]

print(f" Global fallback values calculated for {len(global_fallbacks)} columns")
print(" Sample fallback values:")
for i, (col, val) in enumerate(list(global_fallbacks.items())[:5]):
    print(f"   {col}: {val}")


 Global fallback values calculated for 4 columns
 Sample fallback values:
   customer_zip_code_prefix: 35630.0
   customer_id: rPgpYIdxiO5G
   customer_city: sao paulo
   customer_state: SP


In [7]:
def smart_local_impute(chunk, global_fallbacks):
    numeric_cols = chunk.select_dtypes(include='number').columns
    categorical_cols = chunk.select_dtypes(include='object').columns
    
    for col in numeric_cols:
        if chunk[col].isnull().any():
            non_null = chunk[col].dropna()
            
            if len(non_null) >= 5:  
                fill_value = non_null.median()
            else:
                fill_value = global_fallbacks.get(col, 0)
            
            chunk[col].fillna(fill_value, inplace=True)
    
    for col in categorical_cols:
        if chunk[col].isnull().any():
            non_null = chunk[col].dropna()
            
            if len(non_null) >= 3:  
                mode_series = non_null.mode()
                fill_value = mode_series[0] if not mode_series.empty else "Unknown"
            else:
                fill_value = global_fallbacks.get(col, "Unknown")
            
            chunk[col].fillna(fill_value, inplace=True)
    
    return chunk

print(" Local imputation function defined")

 Local imputation function defined


In [8]:
print(f"\n🔄 Pass 2: Processing {total_chunks} chunks with LOCAL imputation...")

Path(OUTPUT_PATH).unlink(missing_ok=True)

first_write = True
total_output_rows = 0
chunk_stats = []

for chunk_num, chunk in enumerate(pd.read_csv(FILE_PATH, chunksize=CHUNK_SIZE), 1):
    
    original_shape = chunk.shape
    
    chunk_clean = chunk.drop(columns=cols_to_drop, errors='ignore')
    
    row_missing_ratio = chunk_clean.isnull().mean(axis=1)
    chunk_clean = chunk_clean[row_missing_ratio < 0.95]
    
    chunk_clean = smart_local_impute(chunk_clean, global_fallbacks)
    chunk_clean = chunk_clean.drop_duplicates()
    
    if first_write:
        chunk_clean.to_csv(OUTPUT_PATH, index=False, mode='w')
        first_write = False
    else:
        chunk_clean.to_csv(OUTPUT_PATH, index=False, mode='a', header=False)
    
    total_output_rows += len(chunk_clean)
    chunk_stats.append({
        'chunk': chunk_num,
        'original_rows': original_shape[0],
        'final_rows': len(chunk_clean),
        'row_reduction': (original_shape[0] - len(chunk_clean)) / original_shape[0]
    })
    
    if chunk_num % 20 == 0 or chunk_num <= 5:
        print(f" Chunk {chunk_num:3d}: {original_shape[0]:,} → {len(chunk_clean):,} rows")

print(f" All {total_chunks} chunks processed!")



🔄 Pass 2: Processing 9 chunks with LOCAL imputation...
 Chunk   1: 10,000 → 10,000 rows
 Chunk   2: 10,000 → 10,000 rows
 Chunk   3: 10,000 → 10,000 rows
 Chunk   4: 10,000 → 10,000 rows
 Chunk   5: 10,000 → 10,000 rows
 All 9 chunks processed!


In [9]:
final_df_sample = pd.read_csv(OUTPUT_PATH, nrows=1)
final_cols = len(final_df_sample.columns)
original_cols = len(col_missing_counts)

total_original_rows = sum(stat['original_rows'] for stat in chunk_stats)
row_reduction_pct = ((total_original_rows - total_output_rows) / total_original_rows * 100)
col_reduction_pct = ((original_cols - final_cols) / original_cols * 100)

print("🎉 SUCCESS! Smart cleaning completed:")
print("=" * 50)
print(f" ROWS:")
print(f"   Original: {total_original_rows:,}")
print(f"   Final:    {total_output_rows:,}")
print(f"   Reduction: {row_reduction_pct:.1f}%")
print()
print(f" COLUMNS:")
print(f"   Original: {original_cols}")
print(f"   Final:    {final_cols}")
print(f"   Dropped:  {len(cols_to_drop)} ({col_reduction_pct:.1f}%)")
print()
print(f" OUTPUT: {OUTPUT_PATH}")


🎉 SUCCESS! Smart cleaning completed:
 ROWS:
   Original: 89,316
   Final:    89,316
   Reduction: 0.0%

 COLUMNS:
   Original: 4
   Final:    4
   Dropped:  0 (0.0%)

 OUTPUT: ../ml/moretoo/Ecommerce/train/cleaned_Customers.csv


In [10]:
print("🔍 Quality Check - Sampling cleaned data:")

sample_df = pd.read_csv(OUTPUT_PATH, nrows=1000)

print(f" Sample Info:")
print(f"   Shape: {sample_df.shape}")
print(f"   Missing values: {sample_df.isnull().sum().sum()}")
print(f"   Duplicates: {sample_df.duplicated().sum()}")

print(f" Column types:")
print(f"   Numeric: {len(sample_df.select_dtypes(include='number').columns)}")
print(f"   Categorical: {len(sample_df.select_dtypes(include='object').columns)}")
print(f"   Datetime: {len(sample_df.select_dtypes(include='datetime').columns)}")

print(f"Sample columns:")
for i, col in enumerate(sample_df.columns[:10]):
    dtype = sample_df[col].dtype
    null_count = sample_df[col].isnull().sum()
    print(f"   {i+1:2d}. {col:<25} ({dtype}, {null_count} nulls)")

if len(sample_df.columns) > 10:
    print(f"   ... and {len(sample_df.columns) - 10} more columns")


print("\n🚀 SIMPLE SMART CLEANING COMPLETE!")

print("📈 Results:")
print(f"   • Processed {total_chunks} chunks")
print(f"   • Reduced rows by {row_reduction_pct:.1f}%")
print(f"   • Reduced columns by {col_reduction_pct:.1f}%")
print(f"   • Preserved {len(keep_reasons)} important columns")
print()


🔍 Quality Check - Sampling cleaned data:
 Sample Info:
   Shape: (1000, 4)
   Missing values: 0
   Duplicates: 0
 Column types:
   Numeric: 1
   Categorical: 3
   Datetime: 0
Sample columns:
    1. customer_id               (object, 0 nulls)
    2. customer_zip_code_prefix  (int64, 0 nulls)
    3. customer_city             (object, 0 nulls)
    4. customer_state            (object, 0 nulls)

🚀 SIMPLE SMART CLEANING COMPLETE!
📈 Results:
   • Processed 9 chunks
   • Reduced rows by 0.0%
   • Reduced columns by 0.0%
   • Preserved 4 important columns

